In [2]:
from Bio.Seq import Seq
from Bio import SeqIO
from functools import reduce
import os
from multiprocessing.dummy import Pool as MyPool
from time import sleep, ctime
import pandas as pd
import numpy as np
from collections import Counter
import re

### 函数定义

In [2]:
def GetSeq(hg38_dict, chr, start):
    DNA = hg38_dict[chr][int(start) - 1]
    #	DNA = ''.join([eachnt for eachnt in hg38_dict[chr][int(start)-1:int(end)]])
    return DNA


def Translate(DNA, stand):
    """
	功能： DNA翻译为蛋白质氨基酸序列
	输入： DNA序列，strand方向
	输出： 蛋白质氨基酸序列
	注意： 输入的DNA序列必须为完整的，且必须包含起始子ATG
	"""
    DNA = Seq(DNA.upper())
    if stand == '-':
        DNA = DNA.reverse_complement()
    CDS = DNA[DNA.find('ATG'):]
    if len(CDS) == 0:
        return 'No seq'
    elif len(CDS) == 1:
        return 'No Promoter'
    else:
        AAseq = CDS.translate(to_stop=True)
        if len(AAseq) * 3 == len(CDS):
            return 'No end'
        else:
            return ''.join([each for each in AAseq])


def Cal_pos(list1, list2, fuhao='-'):
    """
	根据链方向合并或计算两个位置列表的差集。
	参数：list1，位置列表1；list2，位置列表2；fuhao，链方向。
	返回：根据链方向计算得到的新位置列表。
	注意：list1和list2的格式为[(start1,end1),(start2,end2),...]，两个位置列表表示 DNA 序列在染色体上的位置区间
	"""
    if fuhao == '+':
        for eachlist2 in list2:
            if int(eachlist2[0]) > int(list1[0][0]) and int(eachlist2[1]) < int(list1[-1][1]):
                list1 = list1 + [eachlist2]
        return list1

    elif fuhao == '-':
        list1 = reduce(lambda x, y: x + y,
                       [list(range(min(int(each[0]), int(each[1])), max(int(each[0]), int(each[1])) + 1)) for each in
                        list1])
        list2 = reduce(lambda x, y: x + y,
                       [list(range(min(int(each[0]), int(each[1])), max(int(each[0]), int(each[1])) + 1)) for each in
                        list2])
        return sorted(list(set([each for each in list1 if each not in list2])))
    else:
        return sorted(
            list(set(reduce(lambda x, y: x + y, [list(range(int(each[0]), int(each[1]) + 1)) for each in list1]))))



def qie(seq, n=9):
    """
	功能：将序列按照指定长度切割为不重叠的子序列。
	输入：seq，序列；n，切割的长度，默认为9。
	输出：长度为n的不重叠子序列的集合。
	示例：qie('ABCDEF',3)  
	返回：{'ABC','BCD', 'CDE','DEF'}
	注意：n必须大于0，否则返回空集合。
	"""
    if len(seq) >= n:
        return set([seq[each:each + n] for each in range(len(seq) - n + 1)])
    else:
        return set([])


def cutseq(seq1, seq2, AS_type):
    if seq2 == 'NotFound':
        return []
    if (AS_type in ['SE', 'A5SS', 'A3SS', 'MXE']):
        return list(qie(seq1) - qie(seq2))
    else:
        return list(qie(seq2) - qie(seq1))

def Merge(Max_Gene_dict, row_data):

    # 获取基因的转录本ID
    gene = row_data['geneSymbol']
    Rid = Max_Gene_dict[gene]['Rid']
    # 剪接类型
    AS_type = row_data['splicing_type']
    
    # 1.处理可变剪接事件类型为SE的情况
    if AS_type in ['SE', 'A5SS', 'A3SS']:
        if AS_type == 'SE' :
            # 获取待删除外显子的位置列表
            drop_pos = [(int(x), int(y)) for x, y in [row_data[['exonStart_0base', 'exonEnd']].tolist()]]
        elif AS_type == 'A5SS':
            # 获取待删除外显子区段的位置列表
            drop_pos = [(int(x), int(y)) for x, y in [row_data[['shortEE', 'longExonEnd']].tolist()]]
        elif AS_type == 'A3SS':
            # 获取待删除外显子区段的位置列表
            drop_pos = [(int(x), int(y)) for x, y in [row_data[['longExonStart_0base', 'shortES']].tolist()]]            
       
        # 获取参考外显子的位置列表
        ref_pos = [Max_Gene_dict[gene][eachexon][1:] for eachexon in Max_Gene_dict[gene]['CDS'][1:]]
        # 获取链方向
        stand = Max_Gene_dict[gene]['CDS'][0]
        # 获取染色体名称
        chr = Max_Gene_dict[gene][Max_Gene_dict[gene]['CDS'][1]][0]
        # 计算参考DNA序列
        Ref_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(ref_pos, [[1, 2]], 'xx')])
        # 计算突变DNA序列
        Mutation_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(ref_pos, drop_pos, '-')])

        # 确保 DNA 序列长度为三的倍数，或者添加一个终止密码子
        #if len(Ref_dna) % 3 != 0:
            #Ref_dna = Ref_dna[:-(len(Ref_dna) % 3)]  # 截断为三的倍数长度
         #   Ref_dna += 'N' * (3 - len(Ref_dna) % 3)  # 或者：添加终止密码子
        #if len(Mutation_dna) % 3 != 0:
            #Mutation_dna = Mutation_dna[:-(len(Mutation_dna) % 3)]  # 截断为三的倍数长度
         #   Mutation_dna += 'N' * (3 - len(Mutation_dna) % 3)  # 或者：添加终止密码子
        
        # 翻译参考DNA序列为参考蛋白质序列
        Ref_aa = Translate(Ref_dna, stand)
        # 翻译突变DNA序列为突变蛋白质序列
        Mutation_aa = Translate(Mutation_dna, stand)

        return Rid, Ref_dna, Mutation_dna, Ref_aa, Mutation_aa
        
    # 2.对于内含子滞留（RI）的可变剪接事件类型：
    elif AS_type == 'RI':
        # 获取待添加内含子的位置列表
        add_pos = [(int(x), int(y)) for x, y in [row_data[['riExonStart_0base', 'riExonEnd']].tolist()]]
        # 获取参考外显子的位置列表
        ref_pos = [Max_Gene_dict[gene][eachexon][1:] for eachexon in Max_Gene_dict[gene]['CDS'][1:]]
        # 获取链方向
        stand = Max_Gene_dict[gene]['CDS'][0]
        # 获取染色体名称
        chr = Max_Gene_dict[gene][Max_Gene_dict[gene]['CDS'][1]][0]
        # 计算参考DNA序列，包括待添加内含子的位置
        Ref_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(ref_pos, [[1, 2]], 'xx')])
        # 计算突变DNA序列，去除待添加内含子的位置
        Mutation_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(Cal_pos(ref_pos, add_pos, '+'), [], 'x')])

        # 确保 DNA 序列长度为三的倍数，或者添加一个终止密码子
        #if len(Ref_dna) % 3 != 0:
            #Ref_dna = Ref_dna[:-(len(Ref_dna) % 3)]  # 截断为三的倍数长度
         #   Ref_dna += 'N' * (3 - len(Ref_dna) % 3)  # 或者：添加终止密码子
        #if len(Mutation_dna) % 3 != 0:
            #Mutation_dna = Mutation_dna[:-(len(Mutation_dna) % 3)]  # 截断为三的倍数长度
         #   Mutation_dna += 'N' * (3 - len(Mutation_dna) % 3)  # 或者：添加终止密码子
        
        # 根据链方向翻译参考和突变的DNA序列为蛋白质序列
        Ref_aa = Translate(Ref_dna, stand)
        Mutation_aa = Translate(Mutation_dna, stand)

        return Rid, Ref_dna, Mutation_dna, Ref_aa, Mutation_aa

    # 3.对于互斥的外显子（MXE）的可变剪接事件类型：
    elif AS_type == 'MXE':
        # 获取两个互斥外显子的位置列表
        alt_pos1 = [(int(x), int(y)) for x, y in [row_data[['1stExonStart_0base', '1stExonEnd']].tolist()]]
        alt_pos2 = [(int(x), int(y)) for x, y in [row_data[['2ndExonStart_0base', '2ndExonEnd']].tolist()]]
        
        # 获取参考外显子的位置列表
        ref_pos = [Max_Gene_dict[gene][eachexon][1:] for eachexon in Max_Gene_dict[gene]['CDS'][1:]]
        # 获取链方向、染色体名称
        stand = Max_Gene_dict[gene]['CDS'][0]
        chr = Max_Gene_dict[gene][Max_Gene_dict[gene]['CDS'][1]][0]
        # 计算互斥外显子的全集位置
        All_dna = Cal_pos(ref_pos, alt_pos1 + alt_pos2, '+')
        # 计算两个互斥外显子的突变DNA序列
        Mutation1_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos1, '-')])
        Mutation2_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos2, '-')])
        
        # 确保 DNA 序列长度为三的倍数，或者添加一个终止密码子
        #if len(Mutation1_dna) % 3 != 0:
            #Mutation1_dna = Mutation1_dna[:-(len(Mutation1_dna) % 3)]  # 截断为三的倍数长度
         #   Mutation1_dna += 'N' * (3 - len(Mutation1_dna) % 3)  # 或者：添加终止密码子
        #if len(Mutation2_dna) % 3 != 0:
            #Mutation2_dna = Mutation2_dna[:-(len(Mutation2_dna) % 3)]  # 截断为三的倍数长度
         #   Mutation2_dna += 'N' * (3 - len(Mutation2_dna) % 3)  # 或者：添加终止密码子
        
        # 根据链方向翻译两个互斥外显子的突变DNA序列为蛋白质序列
        Mutation1_aa = Translate(Mutation1_dna, stand)
        Mutation2_aa = Translate(Mutation2_dna, stand)
        
        return Rid, Mutation1_dna, Mutation2_dna, Mutation1_aa, Mutation2_aa
  

### 1. 提取序列信息hg38.fa

In [3]:
# 1. 提取序列信息hg38.fa
hg38_dict = {}
for seq_record in SeqIO.parse("/data3/lishuhan/fasta_gtf/Homo_sapiens.GRCh38.dna.primary_assembly.fa", "fasta"):
    hg38_dict[seq_record.id.strip('chr')] = seq_record
# 判断染色体5是否为纯N
chromosome_5_sequence = hg38_dict['5'].seq
is_all_N = all(base == 'N' for base in chromosome_5_sequence)
print(is_all_N)
# 查看其中一个序列
hg38_dict['5'].seq

False


Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')

In [4]:
chromosome_5_sequence = hg38_dict['5'].seq
# 统计非 'N' 的数量
non_N_count = sum(base != 'N' for base in chromosome_5_sequence)
# 计算非 'N' 的比例
non_N_ratio = non_N_count / len(chromosome_5_sequence)
print("非 'N' 的数量:", non_N_count)
print("非 'N' 的比例:", non_N_ratio)

非 'N' 的数量: 181265378
非 'N' 的比例: 0.9984968402721104


### 2.提取注释信息hg38.gtf

In [5]:
# 查看前两行
h38gtf_file_path ="/data3/lishuhan/fasta_gtf/Homo_sapiens.GRCh38.84.gtf"
with open(h38gtf_file_path, 'r') as file:
    for i, line in enumerate(file):
        if i < 2:
            columns = line.strip().split('\t')
            print(f"Line {i + 1}: {columns}, Number of elements: {len(columns)}")
        else:
            break

Line 1: ['1', 'havana', 'gene', '11869', '14409', '.', '+', '.', 'gene_id "ENSG00000223972"; gene_version "5"; gene_name "DDX11L1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene"; havana_gene "OTTHUMG00000000961"; havana_gene_version "2";'], Number of elements: 9
Line 2: ['1', 'havana', 'transcript', '11869', '14409', '.', '+', '.', 'gene_id "ENSG00000223972"; gene_version "5"; transcript_id "ENST00000456328"; transcript_version "2"; gene_name "DDX11L1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene"; havana_gene "OTTHUMG00000000961"; havana_gene_version "2"; transcript_name "DDX11L1-002"; transcript_source "havana"; transcript_biotype "processed_transcript"; havana_transcript "OTTHUMT00000362751"; havana_transcript_version "1"; tag "basic"; transcript_support_level "1";'], Number of elements: 9


In [6]:
# 统计gtf文件中序列（feature）类型的频数
type_counter = Counter()
# 遍历GTF文件中的每一行
for line in open(h38gtf_file_path, 'r'):
    # 提取第三列的元素
    type_element = line.strip().split('\t')[2]
    type_counter[type_element] += 1
# 打印各种元素出现的频数
for element, count in type_counter.items():
    print(f"{element}: {count}")

gene: 60675
transcript: 199184
exon: 1176808
CDS: 702410
start_codon: 82403
stop_codon: 73802
five_prime_utr: 142040
three_prime_utr: 131709
Selenocysteine: 119


In [7]:
# 提取全部基因的所有转录本的CDS为一个字典型数据
# 即type_list列为'CDS', 'stop_codon', 'start_codon'的所有数据
chr_list = [str(each) for each in range(1, 23)] + ['X', 'Y']
type_list = ['CDS', 'stop_codon', 'start_codon']
Gene_dict = {}

# 遍历GTF文件中的每一行
for line in open(h38gtf_file_path, 'r'):
    # 提取第九列的属性信息
    attributes = line.strip().split('\t')[8].strip(';')
    # 使用正则表达式通过空格或分号拆分属性，忽略引号内的空格
    fields = dict(re.findall(r'(\S+)\s+"([^"]+)"', attributes))

    # 从行中提取相关信息
    gene_id = fields['gene_id']
    transcript_id = fields.get('transcript_id', '')
    exon_number = fields.get('exon_number', '')
    strand = line.strip().split('\t')[6]
    chr = line.strip().split('\t')[0].strip('chr')
    seq_type = line.strip().split('\t')[2]
    start = int(line.strip().split('\t')[3])
    end = int(line.strip().split('\t')[4])

    # 跳过非基因条目 
    if 'gene_name' not in fields:
        continue

    if (chr not in chr_list) or (seq_type not in type_list):
        continue

    if (seq_type == "start_codon"):
        seq_type = "startcodon"
    if (seq_type == "stop_codon"):
        seq_type = "stopcodon" 
    
    gene_name = fields['gene_name']

    #  Gene_dict.setdefault(gene_name, {}) 
    # 创建一个新的键值对，键是 gene_name，对应的值是一个空字典 {}。如果前一行的 gene_name 和当前行相同，它不会删除前一行记录的信息，
    # 而是在原有的基因条目上继续添加新的信息。这样可以确保一个基因名下包含了所有相关的转录本和外显子信息。
    Gene_dict.setdefault(gene_name, {})   
    if transcript_id:
        Gene_dict[gene_name].setdefault(transcript_id, {'CDS': [strand]})  # 同上 Gene_dict.setdefault(gene_name, {})
        CDSID = '_'.join([str(chr), str(seq_type), str(start), str(end), str(strand), str(exon_number)])   # +exon_number， 
        Gene_dict[gene_name][transcript_id]['CDS'].append(CDSID)
        Gene_dict[gene_name][transcript_id][CDSID] = [chr, start, end] # max(int(start), last_n + 1), end]
        #last_n = int(end)


In [8]:
# 查看基因数量
len(Gene_dict)

20206

In [9]:
# 输出 Gene_dict 的前几个键值对
for gene_name, transcripts in list(Gene_dict.items())[11:12]:
    print(f"Gene: {gene_name}")
    for transcript_id, info in transcripts.items():
        print(f"  Transcript: {transcript_id}")
        print(f"    CDS: {info['CDS']}")
        for cds_id, cds_info in info.items():
            if cds_id != 'CDS':
                print(f"    {cds_id}: {cds_info}")
    print("\n")


Gene: ISG15
  Transcript: ENST00000624697
    CDS: ['+', '1_CDS_1014005_1014475_+_3', '1_startcodon_1014005_1014007_+_3', '1_stopcodon_1014476_1014478_+_3']
    1_CDS_1014005_1014475_+_3: ['1', 1014005, 1014475]
    1_startcodon_1014005_1014007_+_3: ['1', 1014005, 1014007]
    1_stopcodon_1014476_1014478_+_3: ['1', 1014476, 1014478]
  Transcript: ENST00000624652
    CDS: ['+', '1_CDS_1014005_1014435_+_3', '1_startcodon_1014005_1014007_+_3']
    1_CDS_1014005_1014435_+_3: ['1', 1014005, 1014435]
    1_startcodon_1014005_1014007_+_3: ['1', 1014005, 1014007]
  Transcript: ENST00000379389
    CDS: ['+', '1_CDS_1013574_1013576_+_1', '1_startcodon_1013574_1013576_+_1', '1_CDS_1013984_1014475_+_2', '1_stopcodon_1014476_1014478_+_2']
    1_CDS_1013574_1013576_+_1: ['1', 1013574, 1013576]
    1_startcodon_1013574_1013576_+_1: ['1', 1013574, 1013576]
    1_CDS_1013984_1014475_+_2: ['1', 1013984, 1014475]
    1_stopcodon_1014476_1014478_+_2: ['1', 1014476, 1014478]




In [10]:
# 过滤没有‘startcodon’和'stopcodon'的转录本
Gene_dict_fil = {}
for eachgene in Gene_dict:
    # 初始化变量 exon_len 记录最长的编码序列长度，start 记录最小的起始位置
    exon_len = 0
    start = 9999999999999999999999999999999999999999999999
    # 遍历当前基因的每个转录本
    for eachtran in Gene_dict[eachgene]:
        # 获取转录本的编码方向和当前转录本的所有CDS信息
        stand = Gene_dict[eachgene][eachtran]['CDS'][0]
        AllCDS = Gene_dict[eachgene][eachtran]['CDS'][1:]
        # 查看当前转录本的所有seq类型
        seqtype_ls = [each2.split('_')[1] for each2 in AllCDS]
        # 判断‘startcodon’和'stopcodon'是否都在seqtype_ls中
        if 'startcodon' in seqtype_ls and 'stopcodon' in seqtype_ls:
            Gene_dict_fil.setdefault(eachgene, {})
            Gene_dict_fil[eachgene][eachtran] = Gene_dict[eachgene][eachtran]


In [11]:
# 查看前后变化
# 转录本个数
print(len(Gene_dict['FAM227B']))
print(len(Gene_dict_fil['FAM227B']))
# 过滤后的基因数量
print(len(Gene_dict))
len(Gene_dict_fil)

7
5
20206


19485

In [12]:
# 遍历 Gene_dict_fil 中的每个基因及其转录本，找到每个基因编码蛋白质最长的转录本，并记录其相关信息到 Max_Gene_dict 字典中
Max_Gene_dict = {}
for eachgene in Gene_dict_fil:
    # 初始化变量 exon_len 记录最长的编码序列长度，start 记录最小的起始位置
    exon_len = 0
    start = 9999999999999999999999999999999999999999999999
    # 遍历当前基因的每个转录本
    for eachtran in Gene_dict_fil[eachgene]:
        # 获取转录本的编码方向和当前转录本的所有CDS信息
        stand = Gene_dict_fil[eachgene][eachtran]['CDS'][0]
        AllCDS = Gene_dict_fil[eachgene][eachtran]['CDS'][1:]
        # 计算当前转录本的编码序列长度
        LenCDS = sum([int(each2.split('_')[3]) - int(each2.split('_')[2]) for each2 in AllCDS])

        # 获取编码序列的起始位置
        if stand == '+':
            startexon = Gene_dict_fil[eachgene][eachtran]['CDS'][1]
            startpos = Gene_dict_fil[eachgene][eachtran][startexon][1]
        else:
            startexon = Gene_dict_fil[eachgene][eachtran]['CDS'][-1]
            startpos = Gene_dict_fil[eachgene][eachtran][startexon][2]
        
        # 如果当前转录本的起始位置比已记录的最小位置还小，则更新 Max_Gene_dict 中当前基因的记录为当前转录本
        if int(stand + str(startpos)) < start:
            start = int(stand + str(startpos))
            Max_Gene_dict[eachgene] = Gene_dict_fil[eachgene][eachtran]
            Max_Gene_dict[eachgene]['Rid'] = eachtran
            exon_len = LenCDS
        # 如果当前转录本的起始位置与已记录的最小位置相同，但编码序列长度比已记录的最长长度还长，则更新 Max_Gene_dict 中当前基因的记录为当前转录本
        elif int(stand + str(startpos)) == start and LenCDS > exon_len:
            exon_len = LenCDS
            Max_Gene_dict[eachgene] = Gene_dict_fil[eachgene][eachtran]
            Max_Gene_dict[eachgene]['Rid'] = eachtran


In [13]:
len(Max_Gene_dict)

19485

In [14]:
# 查看几个
for gene_id in list(Max_Gene_dict.keys())[1:3]:
    gene_data = Max_Gene_dict.get(gene_id)
    print("Gene ID:", gene_id)
    for key, value in gene_data.items():
        print(f"{key}: {value}")
    print()

Gene ID: FO538757.3
CDS: ['+', '1_CDS_182709_182746_+_1', '1_startcodon_182709_182711_+_1', '1_CDS_183114_183240_+_2', '1_CDS_183922_184155_+_3', '1_stopcodon_184156_184158_+_3']
1_CDS_182709_182746_+_1: ['1', 182709, 182746]
1_startcodon_182709_182711_+_1: ['1', 182709, 182711]
1_CDS_183114_183240_+_2: ['1', 183114, 183240]
1_CDS_183922_184155_+_3: ['1', 183922, 184155]
1_stopcodon_184156_184158_+_3: ['1', 184156, 184158]
Rid: ENST00000624431

Gene ID: FO538757.2
CDS: ['-', '1_CDS_195263_195411_-_1', '1_startcodon_195409_195411_-_1', '1_CDS_188791_188902_-_2', '1_CDS_188489_188584_-_3', '1_CDS_188439_188486_-_4', '1_CDS_188126_188266_-_5', '1_CDS_187755_187886_-_6', '1_CDS_187376_187577_-_7', '1_CDS_187129_187287_-_8', '1_CDS_186317_186469_-_9', '1_CDS_185491_185559_-_10', '1_CDS_185220_185350_-_11', '1_stopcodon_185217_185219_-_11']
1_CDS_195263_195411_-_1: ['1', 195263, 195411]
1_startcodon_195409_195411_-_1: ['1', 195409, 195411]
1_CDS_188791_188902_-_2: ['1', 188791, 188902]
1_CDS

### 3.导入rMats结果分析提取可变剪接异构体

In [31]:
# 读取rMats结果
rMats_results = pd.read_csv("/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/rMAts_results.csv")
# 查看原始可变剪接结果
print(rMats_results.shape)
print(rMats_results.columns)
rMats_results.loc[200:203, ]

(172521, 27)
Index(['ID', 'splicing_type', 'GeneID', 'geneSymbol', 'chr', 'strand',
       'PValue', 'FDR', 'IncLevelDifference', 'exonStart_0base', 'exonEnd',
       'upstreamES', 'upstreamEE', 'downstreamES', 'downstreamEE',
       'longExonStart_0base', 'longExonEnd', 'shortES', 'shortEE',
       'flankingES', 'flankingEE', '1stExonStart_0base', '1stExonEnd',
       '2ndExonStart_0base', '2ndExonEnd', 'riExonStart_0base', 'riExonEnd'],
      dtype='object')


,ID,splicing_type,GeneID,geneSymbol,chr,strand,PValue,FDR,IncLevelDifference,exonStart_0base,...,shortES,shortEE,flankingES,flankingEE,1stExonStart_0base,1stExonEnd,2ndExonStart_0base,2ndExonEnd,riExonStart_0base,riExonEnd
200,SE209,SE,ENSG00000160218,TRAPPC10,chr21,+,1.0,1.0,0.004,44063537.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,SE210,SE,ENSG00000160218,TRAPPC10,chr21,+,1.0,1.0,0.003,44074323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,SE211,SE,ENSG00000160218,TRAPPC10,chr21,+,1.0,1.0,-0.000,44077692.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,SE212,SE,ENSG00000160218,TRAPPC10,chr21,+,1.0,1.0,-0.000,44079563.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# 筛选显著的可变剪接事件
rMats_results = rMats_results[(rMats_results['FDR'] < 0.05) & 
                              (abs(rMats_results['IncLevelDifference']) > 0.1)]

# 筛选后的可变剪接事件数量
rMats_results.shape

(13601, 27)

In [33]:
fafile = '/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_MT.fa' # 只有突变后MT的序列文件
fafile2 = '/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_anno.csv' # 序列注释文件
fafile3 = '/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_WT_MT.fa' # 所有突变前和突变后的序列文件
fafile4 = '/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_all_info.csv'  # 包含所有信息文件

fa = open(fafile, 'w') 
fa3 = open(fafile3, 'w')
fa4 = open(fafile4, 'w')

fa2 = open(fafile2, 'w') 
fa2.write('\t'.join(
        ['AS_event_ID', 'AS_type', 'GeneID', 'geneSymbol', 'tranid', 'FDR', 'IncLevelDifference', 'CDS_same', 'dif_cut_seq']) + '\n') 
fa4.write('\t'.join(
        ['AS_event_ID', 'AS_type', 'GeneID', 'geneSymbol', 'tranid', 'FDR', 'IncLevelDifference', 'CDS_same',
         'Unspliced_dna_seq', 'Spliced_dna_seq', 'Unspliced_AA_seq', 'Spliced_AA_seq', 'dif_cut_seq']) + '\n') 


genes_in_dict = list(Max_Gene_dict.keys())

# rMats_results结果逐行提取，并写入结果文件
for i in range(rMats_results.shape[0]):
    row_data = rMats_results.iloc[i]
    geneSymbol = row_data['geneSymbol']
    if geneSymbol not in genes_in_dict:
        continue
    AS_event_ID = row_data['ID']
    AS_type = row_data['splicing_type']
    GeneID = row_data['GeneID']
    FDR = row_data['FDR']
    IncLevelDifference = row_data['IncLevelDifference']


    tranid, Ref_dna, Mutation_dna, Ref_aa, Mutation_aa = Merge(Max_Gene_dict, row_data)

    if Ref_aa == Mutation_aa:
        CDS_same = 'Same'
    else:
        CDS_same = 'Different'
    dif_cut_seq = cutseq(Ref_aa, Mutation_aa, AS_type)
    #print(i)
    # 写入文件
    fa2.write('\t'.join([str(AS_event_ID), str(AS_type), str(GeneID), str(geneSymbol), str(tranid), str(FDR), str(IncLevelDifference), 
                         str(CDS_same), ','.join(map(str, dif_cut_seq))]) + '\n')
    fa4.write('\t'.join([str(AS_event_ID), str(AS_type), str(GeneID), str(geneSymbol), str(tranid), str(FDR), str(IncLevelDifference), 
                         str(CDS_same), str(Ref_dna), str(Mutation_dna), str(Ref_aa), str(Mutation_aa), ','.join(map(str, dif_cut_seq))]) + '\n')


    if CDS_same == 'Different':
        fa3.write('>' + ';'.join([str(AS_event_ID), str(geneSymbol), str(tranid), 'WT']) + '\n' + str(Ref_aa) + '\n')
        fa3.write('>' + ';'.join([str(AS_event_ID), str(geneSymbol), str(tranid), 'MT']) + '\n' + str(Mutation_aa) + '\n')
        fa.write('>' + ';'.join([ str(AS_event_ID), str(geneSymbol), str(tranid), 'MT']) + '\n' + str(Mutation_aa) + '\n')

    else:
        fa3.write('>' + ';'.join([str(AS_event_ID), str(geneSymbol), str(tranid), 'WT']) + '\n' + str(Ref_aa) + '\n')


/data2/lishuhan/mini/lib/python3.7/site-packages/Bio/Seq.py:2808: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [34]:
fafile2 = '/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_anno.csv' # 序列注释文件
DASEs_Peptides_anno = pd.read_csv(fafile2, sep='\t')
DASEs_Peptides_anno.head()

,AS_event_ID,AS_type,GeneID,geneSymbol,tranid,FDR,IncLevelDifference,CDS_same,dif_cut_seq
0,SE127,SE,ENSG00000186866,POFUT2,ENST00000612472,4.363717e-02,0.124,Same,NaN
1,SE222,SE,ENSG00000160218,TRAPPC10,ENST00000291574,3.884862e-02,0.115,Same,NaN
2,SE279,SE,ENSG00000160194,NDUFV3,ENST00000354250,1.012211e-02,-0.115,Different,"EARAEGQLQ,PAPAVLAEE,PKNVVEPKE,VLFTDEGVP,PFRKQG..."
3,SE383,SE,ENSG00000159267,HLCS,ENST00000336648,2.432823e-08,0.220,Same,NaN
4,SE387,SE,ENSG00000159267,HLCS,ENST00000336648,0.000000e+00,0.220,Same,NaN


In [35]:
# 统计突变情况
DASEs_Peptides_anno = pd.read_csv(fafile2, sep='\t')
print(DASEs_Peptides_anno.shape)
print("------------------总体-----------------------")
print(DASEs_Peptides_anno["CDS_same"].value_counts())
print("------------------分类型-----------------------")
counts = DASEs_Peptides_anno.groupby(['AS_type', 'CDS_same']).size()
print(counts)

(12585, 9)
------------------总体-----------------------
Different    7535
Same         5050
Name: CDS_same, dtype: int64
------------------分类型-----------------------
AS_type  CDS_same 
A3SS     Different     109
         Same          682
A5SS     Different     435
         Same          293
MXE      Different    5009
         Same         1052
RI       Different     693
         Same          951
SE       Different    1289
         Same         2072
dtype: int64


### 测试代码，先保留，后续可能使用

In [29]:
# 代码测试
stand = Gene_dict_fil["SAMD11"]['ENST00000622503']['CDS'][0]
AllCDS = Gene_dict_fil["SAMD11"]['ENST00000622503']['CDS'][1:]
# CDSID = '_'.join([chr, exon_number, start, end, strand])  
LenCDS = sum([int(each2.split('_')[3]) - int(each2.split('_')[2]) for each2 in AllCDS])
stand = Gene_dict_fil["SAMD11"]['ENST00000618181']['CDS'][0]
AllCDS = Gene_dict_fil["SAMD11"]['ENST00000618181']['CDS'][1:]
# CDSID = '_'.join([chr, exon_number, start, end, strand])  
LenCDS2 = sum([int(each2.split('_')[3]) - int(each2.split('_')[2]) for each2 in AllCDS])
print(LenCDS)
LenCDS2

2037


1662

In [30]:
##Get thr pos of each tran 提取每个转录本的位置
type_list = ['exon']
Tran_pos_dict = {}
# Tran_pos_dict
for line in open(h38gtf_file_path, 'r'):
    # 提取第九列的属性信息
    attributes = line.strip().split('\t')[8].strip(';')
    # 使用正则表达式通过空格或分号拆分属性，忽略引号内的空格
    fields = dict(re.findall(r'(\S+)\s+"([^"]+)"', attributes))

    # 从行中提取相关信息
    gene_id = fields['gene_id']
    transcript_id = fields.get('transcript_id', '')
    exon_number = fields.get('exon_number', '')
    strand = line.strip().split('\t')[6]
    chr = line.strip().split('\t')[0].strip('chr')
    seq_type = line.strip().split('\t')[2]
    start = int(line.strip().split('\t')[3])
    end = int(line.strip().split('\t')[4])

    # 跳过非基因条目
    if 'gene_name' not in fields:
        continue

    if (chr not in chr_list) or (seq_type not in type_list) or (transcript_id.startswith('NR')):
        continue
    gene_name = fields['gene_name']
    Tran_pos_dict.setdefault(gene_name, {})
    Tran_pos_dict[gene_name].setdefault(transcript_id, {})
    Tran_pos_dict[gene_name][transcript_id].setdefault('start', []).append(start)
    Tran_pos_dict[gene_name][transcript_id].setdefault('end', []).append(end)


In [23]:
# 输出 Tran_pos_dict 的前几个键值对
for gene_name, transcripts in list(Tran_pos_dict.items())[:2]:
    print(f"Gene: {gene_name}")
    for transcript_id, info in transcripts.items():
        print(f"  Transcript: {transcript_id}")
        for cds_id, cds_info in info.items():
            print(f"    {cds_id}: {cds_info}")
    print("\n")

Gene: DDX11L1
  Transcript: ENST00000456328
    start: [11869, 12613, 13221]
    end: [12227, 12721, 14409]
  Transcript: ENST00000450305
    start: [12010, 12179, 12613, 12975, 13221, 13453]
    end: [12057, 12227, 12697, 13052, 13374, 13670]


Gene: WASH7P
  Transcript: ENST00000488147
    start: [29534, 24738, 18268, 17915, 17606, 17233, 16858, 16607, 15796, 15005, 14404]
    end: [29570, 24891, 18366, 18061, 17742, 17368, 17055, 16765, 15947, 15038, 14501]
  Transcript: ENST00000400706
    start: [31878, 26801, 18373, 18037, 17723, 17348, 16969, 16722, 15913, 15085, 14522]
    end: [32015, 26954, 18471, 18183, 17859, 17483, 17170, 16880, 16065, 15153, 14944]




In [31]:
transcripts = Tran_pos_dict['FAM227B']
for transcript_id, info in transcripts.items():
        print(f"  Transcript: {transcript_id}")
        for cds_id, cds_info in info.items():
            print(f"    {cds_id}: {cds_info}")

  Transcript: ENST00000559573
    start: [49396244, 49395975, 49371302, 49331780, 49326962]
    end: [49396442, 49396097, 49371399, 49331849, 49328675]
  Transcript: ENST00000299338
    start: [49620700, 49615121, 49611215, 49589776, 49588016, 49577629, 49576741, 49575011, 49568245, 49541680, 49508211, 49371302, 49367448, 49335419, 49331780, 49328391]
    end: [49620931, 49615243, 49611268, 49590007, 49588083, 49577664, 49576845, 49575109, 49568346, 49541806, 49508348, 49371399, 49367608, 49335496, 49331849, 49328675]
  Transcript: ENST00000560557
    start: [49334218, 49331780, 49328393]
    end: [49334302, 49331849, 49328675]
  Transcript: ENST00000561064
    start: [49620631, 49615121, 49611215, 49589776, 49588016, 49577629, 49576741, 49575011, 49541680, 49508211, 49422400]
    end: [49620929, 49615243, 49611268, 49590007, 49588083, 49577664, 49576845, 49575109, 49541806, 49508348, 49422714]
  Transcript: ENST00000559351
    start: [49508211, 49489061]
    end: [49508294, 49489391]


In [32]:
##Get the exon pos 提取外显子的位置信息
Gene_pos = {}

# 遍历 GTF 文件的每一行
for line in open(h38gtf_file_path, 'r'):
    # 解析每一行的字段
    fields = line.strip().split('\t')
    chr = fields[0].strip('chr')
    seq_type = fields[2]
    start = int(fields[3])
    end = int(fields[4])
    strand = fields[6]

    # 仅处理基因和外显子类型的行
    if seq_type not in ['gene', 'exon']:
        continue

    # 提取基因名和外显子编号
    attributes = fields[8]
    gene_id = re.search(r'gene_id "(.*?)";', attributes).group(1)
    gene_name = re.search(r'gene_name "(.*?)";', attributes).group(1)
    exon_number_match = re.search(r'exon_number "(.*?)";', attributes)
    exon_number = exon_number_match.group(1) if exon_number_match else None

    # 如果是基因行，则创建基因的字典条目
    if seq_type == 'gene':
        Gene_pos.setdefault(gene_name, {})
    # 如果是外显子行，则将外显子的位置信息存储到基因的字典中
    elif seq_type == 'exon':
        if gene_name in Gene_pos:
            Gene_pos[gene_name].setdefault(exon_number, [])
            if strand == '+':
                Gene_pos[gene_name][exon_number] = [start, end]
            else:
                Gene_pos[gene_name][exon_number] = [end, start]


In [28]:
# 输出 Gene_pos 的前几个键值对
for gene_name, exons in list(Gene_pos.items())[:2]:
    print(f"Gene: {gene_name}")
    for exon_number, pos in exons.items():
        print(f"  Exon {exon_number}: {pos}")
    print("\n")

NameError: name 'Gene_pos' is not defined

In [29]:
# 查看rMats结果的可变剪接数据情况
rMatgene = rMats_results['geneSymbol'].unique()
genes_in_dict = list(Gene_dict_fil.keys())
intersection = np.intersect1d(rMatgene, genes_in_dict)  # 交集
print(len(rMatgene))
print(len(genes_in_dict))
print(len(intersection))

4822
19485
4536


In [33]:
gene_data = Max_Gene_dict.get("CLCN7")
print("Gene ID:", "CLCN7")
for key, value in gene_data.items():
    print(f"{key}: {value}")

Gene ID: CLCN7
CDS: ['-', '16_CDS_1474834_1474974_-_1', '16_startcodon_1474972_1474974_-_1', '16_CDS_1465267_1465338_-_2', '16_CDS_1461603_1461674_-_3', '16_CDS_1461405_1461470_-_4', '16_CDS_1460816_1460948_-_5', '16_CDS_1460418_1460527_-_6', '16_CDS_1459107_1459187_-_7', '16_CDS_1457694_1457756_-_8', '16_CDS_1457254_1457337_-_9', '16_CDS_1456113_1456206_-_10', '16_CDS_1455731_1455795_-_11', '16_CDS_1455134_1455250_-_12', '16_CDS_1454411_1454465_-_13', '16_CDS_1453834_1453894_-_14', '16_CDS_1452755_1452893_-_15', '16_CDS_1451623_1451716_-_16', '16_CDS_1450497_1450666_-_17', '16_CDS_1449276_1449327_-_18', '16_CDS_1448966_1449093_-_19', '16_CDS_1448681_1448766_-_20', '16_CDS_1448355_1448484_-_21', '16_CDS_1447655_1447714_-_22', '16_CDS_1447392_1447568_-_23', '16_CDS_1447006_1447086_-_24', '16_CDS_1446634_1446717_-_25', '16_stopcodon_1446631_1446633_-_25']
16_CDS_1474834_1474974_-_1: ['16', 1474834, 1474974]
16_startcodon_1474972_1474974_-_1: ['16', 1474972, 1474974]
16_CDS_1465267_146533

In [34]:
gene = "FAM227B"
AS_type = 'SE'
filt_data = rMats_results[(rMats_results['geneSymbol'] == gene) & (rMats_results['splicing_type'] == AS_type)].iloc[1]
drop_pos = filt_data[['exonStart_0base', 'exonEnd']].tolist()
drop_pos = [(int(drop_pos[0]), int(drop_pos[1]))]  # 转换为包含一个元组的列表
print(drop_pos)
type(drop_pos)

[(49606155, 49606218)]


list

In [116]:
# 读取rMats结果
rMats_results = pd.read_csv("/home/chenweiming/mon_renw/xiaozhi233/data/rMAts_results.csv")
# 筛选显著的可变剪接事件
rMats_results = rMats_results[(rMats_results['FDR'] < 0.05) & 
                              (rMats_results['IncLevelDifference'] < -0.1)]

rMats_results.shape

(8467, 27)

In [36]:
i = 8388
row_data = rMats_results.iloc[i]
geneSymbol = row_data['geneSymbol']
AS_event_ID = row_data['ID']
AS_type = row_data['splicing_type']
GeneID = row_data['GeneID']
FDR = row_data['FDR']
IncLevelDifference = row_data['IncLevelDifference']

tranid, Ref_dna, Mutation_dna, Ref_aa, Mutation_aa = Merge(Max_Gene_dict, row_data)
print(i)
print(Mutation_aa)
# 写入文件
#print(AS_event_ID, '\n', AS_type, '\n', GeneID, '\n', geneSymbol, '\n', str(FDR), '\n', str(IncLevelDifference), '\n', 
#         Ref_dna, '\n', Mutation_dna, '\n', Ref_aa, '\n', Mutation_aa)


8388
MEEGGRDKAPVQPQQSPAAAPGGTDEKPSGKERRDAGDKDKEQELSEEDKQLQDELEMLVERLGEKDTSLYRPALEELRRQIRSSTTSMTSVPKPLKFLRPHYGKLKEIYENMAPGENKRFAADIISVLAMTMSGERECLKYRLVGSQEELASWGHEYVRHLAGEVAKEWQELDDAEKVQREPLLTLVKEIVPYNMAHNAEHEACDLLMEIEQVDMLEKDIDENAYAKVCLYLTSCVNYVPEPENSALLRCALGVFRKFSRFPEALRLALMLNDMELVEDIFTSCKDVVVQKQMAFMLGRHGVFLELSEDVEEYEDLTEIMSNVQLNSNFLALARELDIMEPKVPDDIYKTHLENNRFGGSGSQVDSARMNLASSFVNGFVNAAFGQDKLLTDDGNKWLYKNKDHGMLSAAASLGMILLWDVDGGLTQIDKYLYSSEDYIKSGALLACGIVNSGVRNECDPALALLSDYVLHNSNTMRLGSIFGLGLAYAGSNREDVLTLLLPVMGDSKSSMEVAGVTALACGMIAVGSCNGDVTSTILQTIMEKSETELKDTYARWLPLGLGLNHLGKGEAIEAILAALEVVSEPFRSFANTLVDVCAYAGSGNVLKVQQLLHICSEHFDSKEKEEDKDKKEKKDKDKKEAPADMGAHQGVAVLGIALIAMGEEIGAEMALRTFGHLLRYGEPTLRRAVPLALALISVSNPRLNILDTLSKFSHDADPEVSYNSIFAMGMVGSGTNNARLAAMLRQLAQYHAKDPNNLFMVRLAQGLTHLGKGTLTLCPYHSDRQLMSQVAVAGLLTVLVSFLDVRNIILGKSHYVLYGLVAAMQPRMLVTFDEELRPLPVSVRVGQAVDVVGQAGKPKTITGFQTHTTPVLLAHGERAELATEEFLPVTPILEGFVILRKNPNYDL


In [86]:
i = 1388
row_data = rMats_results.iloc[i]
# 获取基因的转录本ID
gene = row_data['geneSymbol']
Rid = Max_Gene_dict[gene]['Rid']
# 剪接类型
AS_type = row_data['splicing_type']
# 获取两个互斥外显子的位置列表
alt_pos1 = [(int(x), int(y)) for x, y in [row_data[['1stExonStart_0base', '1stExonEnd']].tolist()]]
alt_pos2 = [(int(x), int(y)) for x, y in [row_data[['2ndExonStart_0base', '2ndExonEnd']].tolist()]]

# 获取参考外显子的位置列表
ref_pos = [Max_Gene_dict[gene][eachexon][1:] for eachexon in Max_Gene_dict[gene]['CDS'][1:]]
# 获取链方向、染色体名称
stand = Max_Gene_dict[gene]['CDS'][0]
chr = Max_Gene_dict[gene][Max_Gene_dict[gene]['CDS'][1]][0]
# 计算互斥外显子的全集位置
All_dna = Cal_pos(ref_pos, alt_pos1 + alt_pos2, '+')
# 计算两个互斥外显子的突变DNA序列
Mutation1_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos1, '-')])
Mutation2_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos2, '-')])

# 确保 DNA 序列长度为三的倍数，或者添加一个终止密码子
if len(Mutation1_dna) % 3 != 0:
    #Mutation1_dna = Mutation1_dna[:-(len(Mutation1_dna) % 3)]  # 截断为三的倍数长度
    Mutation1_dna += 'N' * (3 - len(Mutation1_dna) % 3)  # 或者：添加终止密码子
if len(Mutation2_dna) % 3 != 0:
    #Mutation2_dna = Mutation2_dna[:-(len(Mutation2_dna) % 3)]  # 截断为三的倍数长度
    Mutation2_dna += 'N' * (3 - len(Mutation2_dna) % 3)  # 或者：添加终止密码子

# 根据链方向翻译两个互斥外显子的突变DNA序列为蛋白质序列
Mutation1_aa = Translate(Mutation1_dna, stand)
Mutation2_aa = Translate(Mutation2_dna, stand)
#print(AS_event_ID, '\n', AS_type, '\n', GeneID, '\n', geneSymbol, '\n', str(FDR), '\n', str(IncLevelDifference), '\n', Rid, '\n',
#         Mutation1_dna, '\n', Mutation2_dna, '\n', Mutation1_aa, '\n', Mutation1_aa)

In [24]:
# 统计突变情况
output_file_path = '/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_anno.csv'
DASEs_Peptides_anno = pd.read_csv(fafile2, sep='\t')
print(DASEs_Peptides_anno.shape)
print("------------------总体-----------------------")
print(DASEs_Peptides_anno["CDS_same"].value_counts())
print("------------------分类型-----------------------")
counts = DASEs_Peptides_anno.groupby(['AS_type', 'CDS_same']).size()
print(counts)
DASEs_Peptides_anno.to_csv(output_file_path, sep=',', index=False)
print(f"DataFrame has been saved to {output_file_path}")

(7908, 9)
------------------总体-----------------------
Different    4899
Same         3009
Name: CDS_same, dtype: int64
------------------分类型-----------------------
AS_type  CDS_same 
A3SS     Different      56
         Same          492
A5SS     Different     307
         Same          203
MXE      Different    3500
         Same          719
RI       Different     625
         Same          861
SE       Different     411
         Same          734
dtype: int64
DataFrame has been saved to /data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_anno.csv


In [24]:
DASEs_Peptides_anno.head()

,AS_event_ID,AS_type,GeneID,geneSymbol,tranid,FDR,IncLevelDifference,CDS_same,dif_cut_seq
0,SE_279,SE,ENSG00000160194,NDUFV3,ENST00000354250,0.013306,0.115,Different,"GSDSEARQV,SSSSDSESD,QKVLSPFRK,EEFLKQSLK,DKESQK..."
1,SE_424,SE,ENSG00000241837,ATP5O,ENST00000354250,0.004938,0.360,Different,NaN
2,SE_603,SE,ENSG00000154642,C21orf91,ENST00000290299,0.001135,1.000,Same,NaN
3,SE_688,SE,ENSG00000275895,U2AF1L5,ENST00000400558,0.000022,0.113,Same,"LIQNIYRNP,PTFSQTILI,PQNSAQTAD,TADGSHYHC,AQTADG..."
4,SE_754,SE,ENSG00000100239,PPP6R2,ENST00000620065,0.000184,0.306,Different,NaN


In [3]:
df = pd.read_csv("/data3/lishuhan/30GSE_health_SRR/v3AS/v2_old_you/file/DASEs_Peptides_anno.csv")

In [4]:
df.head()

,AS_event_ID,AS_type,GeneID,geneSymbol,tranid,FDR,IncLevelDifference,CDS_same,dif_cut_seq
0,279,SE,ENSG00000160194,NDUFV3,ENST00000354250,0.013306,0.115,Different,"PVHTKSGLS,APPLPRKET,KGPLPVHTK,RVTVSAKEK,LSAPPK..."
1,424,SE,ENSG00000241837,ATP5O,ENST00000290299,0.004938,0.360,Same,NaN
2,603,SE,ENSG00000154642,C21orf91,ENST00000400558,0.001135,1.000,Same,NaN
3,688,SE,ENSG00000275895,U2AF1L5,ENST00000620065,0.000022,0.113,Different,"TADGSHYHC,TILIQNIYR,SHYHCPLEH,HYHCPLEHL,ADGSHY..."
4,754,SE,ENSG00000100239,PPP6R2,ENST00000216061,0.000184,0.306,Same,NaN


In [226]:
print(min(Cal_pos(ref_pos, [[1, 2]], 'xx')))
max(Cal_pos(ref_pos, [[1, 2]], 'xx'))

36053589


36058888

In [42]:
gene = geneSymbol

In [43]:
alt_pos1 = [(int(x), int(y)) for x, y in [row_data[['2ndExonStart_0base', '2ndExonEnd']].tolist()]]

alt_pos1

[(45480466, 45480520)]

In [44]:
# 3.对于互斥的外显子（ME）的可变剪接事件类型：
# elif type == 'MXE':
# 获取两个互斥外显子的位置列表
alt_pos1 = [(int(x), int(y)) for x, y in [row_data[['1stExonStart_0base', '1stExonEnd']].tolist()]]
alt_pos2 = [(int(x), int(y)) for x, y in [row_data[['2ndExonStart_0base', '2ndExonEnd']].tolist()]]

# 获取参考外显子的位置列表
ref_pos = [Max_Gene_dict[gene][eachexon][1:] for eachexon in Max_Gene_dict[gene]['CDS'][1:]]
# 获取链方向、染色体名称
stand = Max_Gene_dict[gene]['CDS'][0]
chr = Max_Gene_dict[gene][Max_Gene_dict[gene]['CDS'][1]][0]
# 计算互斥外显子的全集位置
All_dna = Cal_pos(ref_pos, alt_pos1 + alt_pos2, '+')
# 计算两个互斥外显子的突变DNA序列
Mutation1_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos1, '-')])
Mutation2_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos2, '-')])

# 确保 DNA 序列长度为三的倍数，或者添加一个终止密码子
if len(Mutation1_dna) % 3 != 0:
    #Mutation1_dna = Mutation1_dna[:-(len(Mutation1_dna) % 3)]  # 截断为三的倍数长度
    Mutation1_dna += 'N' * (3 - len(Mutation1_dna) % 3)  # 或者：添加终止密码子
if len(Mutation2_dna) % 3 != 0:
    #Mutation2_dna = Mutation2_dna[:-(len(Mutation2_dna) % 3)]  # 截断为三的倍数长度
    Mutation2_dna += 'N' * (3 - len(Mutation2_dna) % 3)  # 或者：添加终止密码子

# 根据链方向翻译两个互斥外显子的突变DNA序列为蛋白质序列
Mutation1_aa = Translate(Mutation1_dna, stand)
Mutation2_aa = Translate(Mutation2_dna, stand)

# return Rid, Rid, Mutation2_dna, Mutation1_dna, Mutation2_aa, Mutation1_aa, A

# 写入文件
print(AS_event_ID, '\n', AS_type, '\n', GeneID, '\n', geneSymbol, '\n', str(FDR), '\n', str(IncLevelDifference), '\n', 
         Mutation1_dna, '\n', Mutation2_dna, '\n', Mutation1_aa, '\n', Mutation1_aa)

MXE_28 
 MXE 
 ENSG00000182871 
 COL18A1 
 7.657053864909363e-15 
 -0.174 
 AtggcgccgagGTGCCCCTGGCCATGGCCGCGGCGGCGGCGCCTCCTGGACGTGCTCGCGCCCCTGGTCCTGCTGCTCGGGGTCCGCGCGGCCTCCGCGGAGCCAGAGCGCATCAGCGAGGAGGTGGGGCTGCTGCAGCTCCTTGGGGACCCCCCGCCCCAGCAGGTCACCCAGACGGATGACCCCGACGTCGGGCTGGCCTACGTCTTTGGGCCAGATGCCAACAGTGGCCAAGTGGCCCGGTACCACTTCCCCAGCCTCTTCTTCCGTGACTTCTCACTGCTGTTCCACATCCGGCCAGCCACAGAGGGCCCAGGGGTGCTGTTCGCCATCACGGACTCGGCGCAGGCCATGGTCTTGCTGGGCGTGAAGCTCTCTGGGGTGCAGGACGGGCACCAGGACATCTCCCTGCTCTACACAGAACCAGGTGCAGGCCAGACCCACACAGCCGCCAGCTTCCGGCTCCCCGCCTTCGTCGGCCAGTGGACACACTTAGCCCTCAGTGTGGCAGGTGGCTTTGTGGCCCTCTACGTGGACTGTGAGGAGTTCCAGAGAATGCCGCTTGCTCGGTCCTCACGGGGCCTGGAGCTGGAGCCTGGCGCCGGGCTCTTCGTGGCTCAGGCGGGGGGAGCGGACCCTGACAAGTTCCAGGGGGTGATCGCTGAGCTGAAGGTGCGCAGGGACCCCCAGGTGAGCCCCATGCACTGCCTGGACGAGGAAGGCGATGACTCAGATGGGGCATCCGGAGACTCTGGCAGCGGGCTCGGGGACGCCCGGGAGCTTCTCAGGGAGGAGACGGGCGCGGCCCTAAAACCCAGGCTCCCCGCGCCACCCCCCGTCACCACGCCACCCTTGGCTGGAGGCAGCAGCACGGAAGATTCCAGAAGTGAAGAAGTCGAGGAGCAGACCACGGTGGCTTCGTTA

In [304]:
# 获取两个突变DNA序列
Mutation1_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos1, '-')])
Mutation2_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos2, '-')])
print(len(Mutation1_dna))
len(Mutation2_dna)

3933


3967

In [314]:
# 获取两个突变DNA序列
Mutation1_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos1, '-')])
Mutation2_dna = ''.join([GetSeq(hg38_dict, chr, eachpos) for eachpos in Cal_pos(All_dna, alt_pos2, '-')])

# 确保 DNA 序列长度为三的倍数，或者添加一个终止密码子
if len(Mutation1_dna) % 3 != 0:
    #Mutation1_dna = Mutation1_dna[:-(len(Mutation1_dna) % 3)]  # 截断为三的倍数长度
    Mutation1_dna += 'N' * (3 - len(Mutation1_dna) % 3)  # 或者：添加终止密码子
if len(Mutation2_dna) % 3 != 0:
    #Mutation2_dna = Mutation2_dna[:-(len(Mutation2_dna) % 3)]  # 截断为三的倍数长度
    Mutation2_dna += 'N' * (3 - len(Mutation2_dna) % 3)  # 或者：添加终止密码子

# 翻译为蛋白质序列
Mutation1_aa = Translate(Mutation1_dna, stand)
Mutation2_aa = Translate(Mutation2_dna, stand)
print(len(Mutation1_dna))
len(Mutation2_dna)

3933


3969

In [307]:
Mutation2_dna

'AtggcgccgagGTGCCCCTGGCCATGGCCGCGGCGGCGGCGCCTCCTGGACGTGCTCGCGCCCCTGGTCCTGCTGCTCGGGGTCCGCGCGGCCTCCGCGGAGCCAGAGCGCATCAGCGAGGAGGTGGGGCTGCTGCAGCTCCTTGGGGACCCCCCGCCCCAGCAGGTCACCCAGACGGATGACCCCGACGTCGGGCTGGCCTACGTCTTTGGGCCAGATGCCAACAGTGGCCAAGTGGCCCGGTACCACTTCCCCAGCCTCTTCTTCCGTGACTTCTCACTGCTGTTCCACATCCGGCCAGCCACAGAGGGCCCAGGGGTGCTGTTCGCCATCACGGACTCGGCGCAGGCCATGGTCTTGCTGGGCGTGAAGCTCTCTGGGGTGCAGGACGGGCACCAGGACATCTCCCTGCTCTACACAGAACCAGGTGCAGGCCAGACCCACACAGCCGCCAGCTTCCGGCTCCCCGCCTTCGTCGGCCAGTGGACACACTTAGCCCTCAGTGTGGCAGGTGGCTTTGTGGCCCTCTACGTGGACTGTGAGGAGTTCCAGAGAATGCCGCTTGCTCGGTCCTCACGGGGCCTGGAGCTGGAGCCTGGCGCCGGGCTCTTCGTGGCTCAGGCGGGGGGAGCGGACCCTGACAAGTTCCAGGGGGTGATCGCTGAGCTGAAGGTGCGCAGGGACCCCCAGGTGAGCCCCATGCACTGCCTGGACGAGGAAGGCGATGACTCAGATGGGGCATCCGGAGACTCTGGCAGCGGGCTCGGGGACGCCCGGGAGCTTCTCAGGGAGGAGACGGGCGCGGCCCTAAAACCCAGGCTCCCCGCGCCACCCCCCGTCACCACGCCACCCTTGGCTGGAGGCAGCAGCACGGAAGATTCCAGAAGTGAAGAAGTCGAGGAGCAGACCACGGTGGCTTCGTTAGGAGCTCAGACACTTCCTGGCTCAGATTCTGTCTCCACGTGGGACGGGAGTGTCCGGACCCCTGGGGGCCGCGTG

In [238]:
Gene_dict[gene]['ENST00000443703']

{'CDS': ['-',
  '21_CDS_36058816_36058888_-_2',
  '21_startcodon_36058886_36058888_-_2',
  '21_CDS_36057109_36057204_-_3',
  '21_CDS_36053589_36053620_-_4'],
 '21_CDS_36058816_36058888_-_2': ['21', 36058816, 36058888],
 '21_startcodon_36058886_36058888_-_2': ['21', 36058886, 36058888],
 '21_CDS_36057109_36057204_-_3': ['21', 36057109, 36057204],
 '21_CDS_36053589_36053620_-_4': ['21', 36053589, 36053620],
 'Rid': 'ENST00000443703'}

In [165]:
transcripts = Tran_pos_dict['FAM227B']
for transcript_id, info in transcripts.items():
        print(f"  Transcript: {transcript_id}")
        for cds_id, cds_info in info.items():
            print(f"    {cds_id}: {cds_info}")

  Transcript: ENST00000559573
    start: [49396244, 49395975, 49371302, 49331780, 49326962]
    end: [49396442, 49396097, 49371399, 49331849, 49328675]
  Transcript: ENST00000299338
    start: [49620700, 49615121, 49611215, 49589776, 49588016, 49577629, 49576741, 49575011, 49568245, 49541680, 49508211, 49371302, 49367448, 49335419, 49331780, 49328391]
    end: [49620931, 49615243, 49611268, 49590007, 49588083, 49577664, 49576845, 49575109, 49568346, 49541806, 49508348, 49371399, 49367608, 49335496, 49331849, 49328675]
  Transcript: ENST00000560557
    start: [49334218, 49331780, 49328393]
    end: [49334302, 49331849, 49328675]
  Transcript: ENST00000561064
    start: [49620631, 49615121, 49611215, 49589776, 49588016, 49577629, 49576741, 49575011, 49541680, 49508211, 49422400]
    end: [49620929, 49615243, 49611268, 49590007, 49588083, 49577664, 49576845, 49575109, 49541806, 49508348, 49422714]
  Transcript: ENST00000559351
    start: [49508211, 49489061]
    end: [49508294, 49489391]


In [105]:
Gene_pos["FAM227B"]

{'1': [49620816, 49620700],
 '2': [49615243, 49614792],
 '3': [49611268, 49611215],
 '4': [49606218, 49606156],
 '5': [49590007, 49589776],
 '6': [49577664, 49577629],
 '7': [49577305, 49577172],
 '8': [49576845, 49576761],
 '9': [49541806, 49541680],
 '10': [49508348, 49508211],
 '11': [49422714, 49422400],
 '12': [49371399, 49371302],
 '13': [49367608, 49367448],
 '14': [49335496, 49335419],
 '15': [49331849, 49331780],
 '16': [49328675, 49328391]}

In [185]:
transcripts = Gene_dict_fil['SETD4']
for transcript_id, info in transcripts.items():
        print(f"  Transcript: {transcript_id}")
        print(f"    CDS: {info['CDS']}")
        for cds_id, cds_info in info.items():
            if cds_id != 'CDS':
                print(f"    {cds_id}: {cds_info}")

  Transcript: ENST00000399215
    CDS: ['-', '21_CDS_36058816_36058888_-_1', '21_startcodon_36058886_36058888_-_1', '21_CDS_36057109_36057204_-_2', '21_CDS_36053583_36053620_-_3', '21_CDS_36048308_36048396_-_4', '21_CDS_36045582_36046011_-_5', '21_CDS_36043782_36043956_-_6', '21_CDS_36041807_36041888_-_7', '21_CDS_36040575_36040655_-_8', '21_CDS_36038150_36038273_-_9', '21_CDS_36036120_36036251_-_10', '21_stopcodon_36036117_36036119_-_10']
    21_CDS_36058816_36058888_-_1: ['21', 36058816, 36058888]
    21_startcodon_36058886_36058888_-_1: ['21', 36058886, 36058888]
    21_CDS_36057109_36057204_-_2: ['21', 36057109, 36057204]
    21_CDS_36053583_36053620_-_3: ['21', 36053583, 36053620]
    21_CDS_36048308_36048396_-_4: ['21', 36048308, 36048396]
    21_CDS_36045582_36046011_-_5: ['21', 36045582, 36046011]
    21_CDS_36043782_36043956_-_6: ['21', 36043782, 36043956]
    21_CDS_36041807_36041888_-_7: ['21', 36041807, 36041888]
    21_CDS_36040575_36040655_-_8: ['21', 36040575, 36040655]
